In [ ]:
# Grafo
edges = {(0, 1): 5, (0, 2): 8, (1, 2): 2, (1, 3): 7, (2, 3): 4}
num_nodes = 4

# Hp = 11*Z0 - 17.5*Z1 - 28*Z2 - 17*Z3 + 11.5*Z4 +
#      13.5*(Z0*Z1 + Z1*Z2 + Z2*Z3 - Z0*Z2 - Z0*Z3 + Z3*Z4 - Z1*Z4 - Z2*Z4)

# Coeficientes lineales de Hp (11 * Z0 - 17.5 * Z1...)
linear_coefs = [11, -17.5, -28, -17, 11.5]

# Coeficientes cuadráticos Ej: {(2, 3): 13.5} -> 13.5 * Z2 * Z3
quadra_coefs = {(0, 1): 13.5, (1, 2): 13.5, (2, 3): 13.5, (0, 2): -13.5,
                (0, 3): -13.5, (3, 4): 13.5, (1, 4): -13.5, (2, 4): -13.5}

In [ ]:
cost_function = "\
5*x_01 + 8*x_02 + 2*x_12 + 7*x_13 + 4*x_23 + \
27*(x_01 + x_02 - 1)**2 + \
27*(x_13 + x_23 - 1)**2 + \
27*(x_01 - x_12 - x_13)**2 + \
27*(x_02 + x_12 - x_23)**2"

# Evalúa una solución concreta según la función de coste para la versión QUBO
# Entrada:
#     - bits: Cadena con los valores de los qubits medidos
#             Orden: q_4 q_3 q_2 q_1 q_0
def eval_cost_function(bits):
    assert len(edges) == len(bits), "Error in cost_function: Length of bits"

    inv_bits = bits[::-1]  # Los qubits están en orden inverso

    param_dict = {}
    # Asignación entre x_ij -> q_n
    # Establecido por el orden de edges. Ej: x_12 -> q_2 (esto es, qubit nº2)
    for n, (i, j) in enumerate(edges):
        param_dict[f"x_{i}{j}"] = int(inv_bits[n])

    return eval(cost_function, param_dict)

In [ ]:
from qiskit import QuantumCircuit
def generate_qaoa_circuit(theta):
    assert len(theta) % 2 == 0, "Error in parameters (Beta, Gamma)"

    nqubits = len(edges)  # Tantos qubits como aristas tenga el grafo
    circuit = QuantumCircuit(nqubits)

    layers = int(len(theta) / 2)
    beta = theta[:layers]
    gamma = theta[layers:]

    # |v0>
    for i in range(nqubits):
        circuit.h(i)

    for p in range(layers):
        circuit.barrier()

        # Hp
        for q_idx, coef in enumerate(linear_coefs):
            #circuit.rz(coef * gamma[p], q_idx)
            circuit.rz(coef, q_idx)
        for q_idxs, coef in quadra_coefs.items():
            circuit.rzz(coef * gamma[p] * 2, q_idxs[0], q_idxs[1])

        circuit.barrier()

        # Hm
        for q_idx in range(nqubits):
            circuit.rx(beta[p] * 2, q_idx)

    circuit.measure_all()
    return circuit

In [ ]:
def compute_expectation(counts):
    media = 0
    len_count = 0
    for bits, count in counts.items():
        cost = eval_cost_function(bits)
        media += cost * count
        len_count += count

    return media/len_count

In [ ]:
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider()
print(provider.backends())

In [ ]:
# Provider
from qiskit import execute
from scipy.optimize import minimize

device = provider.get_backend("ibmq_qasm_simulator")
shots = 512

def execute_circuit(theta):
    qc = generate_qaoa_circuit(theta)
    counts = execute(qc, device, shots=shots).result().get_counts()
    return compute_expectation(counts)

num_layers = 1
res = minimize(execute_circuit, [1.0, 1.0] * num_layers, method = "COBYLA",
               options={'maxiter': 2})
res

In [ ]:
from qiskit.visualization import plot_histogram
from qiskit import execute

device = provider.get_backend("ibmq_qasm_simulator")
shots = 512

qc = generate_qaoa_circuit([1.0, 1.0])
counts = execute(qc, device, shots=shots).result().get_counts()
plot_histogram(counts, figsize=(14, 10))